In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

diabetes = datasets.load_diabetes()
x = diabetes.data
y = diabetes.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [2]:
# 使用牛顿迭代公式计算
#B*= （𝑋^T*X）^−1*𝑋^𝑇*Y
import numpy as np
a = np.ones(x_train.shape)
XX_train = np.c_[x_train, a]
# pinv伪逆=（𝑋^T*X）^−1*𝑋^𝑇
y_mytest = np.c_[x_test,np.ones(x_test.shape)].dot(np.linalg.pinv(XX_train).dot(y_train))
RMSE = ((y_mytest-y_test)**2).sum()
RMSE = np.sqrt(RMSE/y_mytest[0])
print("RMSE for 牛顿迭代: ", RMSE)

RMSE for 牛顿迭代:  42.6749476083012


In [3]:
# 官方实现
from sklearn.linear_model import LinearRegression

clf = LinearRegression(n_jobs=-1)
clf.fit(x_train, y_train)
y_mytest2 = clf.predict(x_test)
RMSE2 = ((y_mytest2-y_test)**2).sum()
RMSE2 = np.sqrt(RMSE2/y_test[0])
print("RMSE for official linear regression：", RMSE2)

RMSE for official linear regression： 36.83891813127772


In [4]:
# 梯度下降法RMSProp
threshold = 50
a = np.ones(len(x_train))
XX_train = np.c_[a, x_train]
row, col = XX_train.shape
beta = np.random.random([col, 1])
mu = 0.9
vt = np.ones([col, 1])
alpha = 0.5
loss = []
for i in range(100000):
    cur_loss = 1/row * np.linalg.norm(y_train.reshape(row,1)-XX_train.dot(beta))**2
    loss.append(cur_loss)
    if cur_loss < threshold:
        break
    else:
        delta_beta = 2/row * (XX_train.T.dot(y_train.reshape(row, 1))-XX_train.T.dot(XX_train).dot(beta))
        new_vt = mu*vt + (1-mu) * delta_beta**2
        beta = beta + alpha * delta_beta/(np.sqrt(new_vt)+np.spacing(1))
print(cur_loss)
y_mytest3 = np.c_[np.ones(len(x_test)), x_test].dot(beta)
RMSE = np.sqrt(1/len(x_test)*np.sum((y_test.reshape(-1,1)-y_mytest3.reshape(-1,1))**2))
print("RMSE for 梯度下降: ", RMSE)

2778.2018480576503
RMSE for 梯度下降:  55.89130682910378
